In [ ]:
import geopandas as gpd
import pandas as pd
import re
from razdel import tokenize
import pymorphy3
from nltk.tokenize import word_tokenize
from difflib import SequenceMatcher

Поиск категорий

In [ ]:
# Load the GeoJSON files
final_gdf = gpd.read_file("final.geojson")
final_gdf

In [ ]:
# Инициализация морфологического анализатора
morph = pymorphy3.MorphAnalyzer()

In [ ]:
# Список русских стоп-слов
russian_stopwords = [
    "и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", 
    "она", "так", "его", "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", 
    "только", "ее", "мне", "было", "вот", "от", "меня", "еще", "нет", "о", "из", 
    "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже", "или", 
    "ни", "быть", "был", "него", "до", "вас", "нибудь", "опять", "уж", "вам", 
    "ведь", "там", "потом", "себя", "ничего", "ей", "может", "они", "тут", "где", 
    "есть", "надо", "ней", "для", "мы", "тебя", "их", "чем", "была", "сам", 
    "чтоб", "без", "будто", "чего", "раз", "тоже", "себе", "под", "будет", "ж", 
    "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем", "ним", 
    "здесь", "этом", "один", "почти", "мой", "тем", "чтобы", "нее", "сейчас", 
    "были", "куда", "зачем", "всех", "никогда", "можно", "при", "наконец", 
    "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через", 
    "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три", 
    "эту", "моя", "впрочем", "хорошо", "свою", "этой", "перед", "иногда", 
    "лучше", "чуть", "том", "нельзя", "такой", "им", "более", "всегда", 
    "конечно", "всю", "между", "это", "ломоносов", "vk", "метро", "европа",
    "парк", "санкт-петербург", "санкт", "петербург", "очень", "здравствуйте", 
    "время", "год", "город", "https", "также", "id", "который", "всё", "весь", "kgainfo"
]


In [ ]:
def preprocessing(text, stop_words=None):
    """
    Препроцессинг текста: токенизация, приведение к нижнему регистру, лемматизация, удаление стоп-слов и пунктуации.
    :param text: Исходный текст.
    :param stop_words: Список стоп-слов для удаления (по умолчанию None).
    :return: Лемматизированный текст.
    """
    if not isinstance(text, str) or text.strip() == "":
        return ""
    # Токенизация с помощью razdel
    tokens = [token.text.lower() for token in tokenize(text)]
    # Лемматизация
    lemms = [morph.parse(token)[0].normal_form for token in tokens if token.isalpha()]
    # Удаление стоп-слов
    if stop_words:
        lemms = [word for word in lemms if word not in stop_words]
    return " ".join(lemms)


In [ ]:
#функция очистки: оставляем только русские символы и удаляем ссылки и то, что не смогли удалить другии путями
def clean(text):
    emoji_pattern = re.compile(pattern = "["
                               u"\U00000000-\U00000009" #управляющие символы (control characters)
                                u"\U0000000B-\U0000001F" #управляющие символы (control characters)
                                u"\U00000041-\U0000007A" #буквы английского алфавита
                                u"\U00000080-\U00000400" #специальные символы и цифры
                                u"\U00000402-\U0000040F" #нерусские символы кириллицы
                                u"\U00000452-\U0010FFFF" #нерусские символы
                                u"/"                     #косая черта вправо
                                u"|"                     #вертикальная черта
                                u":"                     #двоеточие
                                "]+", flags = re.UNICODE)

    url_pattern = re.compile(r'http\S+')

    #удаляем ссылки (URL) из данных
    text_without_urls = url_pattern.sub(r'', text)
    cleaned_text = emoji_pattern.sub(r'', text_without_urls)

    return cleaned_text

In [ ]:
final_gdf["processed_text"] = final_gdf["text"].apply(lambda x: preprocessing(x, stop_words=russian_stopwords))

In [ ]:
final_gdf["processed_text"] = final_gdf["processed_text"].apply(lambda x: clean(x))

In [ ]:
categories = {
    "Функционально-структурные показатели": [
        "удобство", "лавочки", "пруд",
        "детская", "отдых", "дорожки", "фонтан",
        "спорт", "прогулки", "тихо"
    ],
    "Урботехнические показатели": [
        "доступ", "рядом", "транспорт", "парковка",
        "шум", "комфорт", "далеко", "недоступно"
    ],
    "Экологические показатели": [
        "воздух", "дышать", "свежо", "растения",
        "деревья", "кусты", "трава", "зелень",
        "чистый", "грязный", "ухоженный", "гнездо",
        "эстетика", "цветы", "клумбы", "птицы"
    ],
    "Эксплуатационные показатели": [
        "безопасность", "опасно", "свет", "камеры",
        "поломка", "уход", "ремонт", "сломано",
        "чистить", "убирают", "комфорт"
    ]
}

In [ ]:
# Функция для проверки схожести слов
def similar(a, b, threshold=0.9):
    return SequenceMatcher(None, a, b).ratio() >= threshold

In [ ]:
def match_keywords_unique_normalized_fixed(text_words, categories, threshold=0.9):
    """
    Функция подсчёта совпадений с учётом только уникальных совпадений и нормализацией.
    """
    if not text_words:  # Если текст пустой
        return {key: 0 for key in categories.keys()}

    # Словарь для хранения уникальных совпадений
    match_counts = {key: set() for key in categories.keys()}

    for word in text_words:
        for category, keywords in categories.items():
            if any(similar(word, keyword.lower(), threshold) for keyword in keywords):
                match_counts[category].add(word)  # Добавляем слово как уникальное совпадение

    # Нормализация по количеству ключевых слов
    normalized_counts = {
        category: len(matches) / len(categories[category]) if len(categories[category]) > 0 else 0
        for category, matches in match_counts.items()
    }

    print("Уникальные совпадения по категориям:", {k: list(v) for k, v in match_counts.items()})
    print("Нормализованные значения по категориям:", normalized_counts)
    return normalized_counts

In [ ]:
# Применение функции к каждому тексту в столбце processed_text
final_gdf["normalized_counts"] = final_gdf["text"].apply(
    lambda text: match_keywords_unique_normalized_fixed(text.split(), categories, threshold=0.9)
)

# Разделяем словарь на отдельные колонки
normalized_counts_df = pd.DataFrame(final_gdf["normalized_counts"].tolist(), index=final_gdf.index)

# Добавляем новые колонки в DataFrame
final_gdf = pd.concat([final_gdf, normalized_counts_df], axis=1)

# Вывод первых строк для проверки
print(final_gdf.head())

Анализ

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [ ]:
# Колонки с нормализованными значениями категорий
normalized_columns = [
    "Функционально-структурные показатели",
    "Урботехнические показатели",
    "Экологические показатели",
    "Эксплуатационные показатели"
]

# Группируем данные по `label` и рассчитываем средние значения
grouped_data = final_gdf.groupby("label")[normalized_columns].mean()

# Построение графика
ax = grouped_data.plot(kind="bar", figsize=(10, 6), alpha=0.8, cmap="Set2")
ax.set_title("Распределение нормализованных значений по эмоциям", fontsize=16)
ax.set_xlabel("Эмоции", fontsize=12)
ax.set_ylabel("Среднее нормализованное значение", fontsize=12)
plt.xticks(rotation=0)
plt.legend(title="Категории показателей", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# Подготовка данных
X = final_gdf[["Функционально-структурные показатели", "Урботехнические показатели", "Экологические показатели", "Эксплуатационные показатели"]]
y = final_gdf["emotion_score"]

In [ ]:
# Расчет корреляции и p-value для каждого признака
correlations = []

for column in X.columns:
    corr, p_value = pearsonr(X[column], y)  # Вычисляем корреляцию и p-value
    correlations.append({"Feature": column, "Correlation": corr, "P-value": p_value})

# Преобразуем результаты в DataFrame
correlation_df = pd.DataFrame(correlations).sort_values(by="Correlation", ascending=False)

# Выводим результаты
print("Корреляции и p-value каждого признака с целевой переменной:")
correlation_df

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели Random Forest
rf_model = RandomForestRegressor(random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)

# Важность признаков
feature_importances = rf_model.feature_importances_
importance_df = pd.DataFrame({
    "feature": X.columns,
    "importance": feature_importances
}).sort_values(by="importance", ascending=False)

In [ ]:
# Визуализация важности признаков
plt.figure(figsize=(10, 6))
plt.barh(importance_df["feature"], importance_df["importance"], color="skyblue")
plt.xlabel("Важность", fontsize=12)
plt.ylabel("Признаки", fontsize=12)
plt.title("Важность признаков по Random Forest", fontsize=16)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()